![Podmanlogo](Pictures/podman-logo.png)

# Podman Networking

Podman uses the default network "podman" for all containers unless specified differently. If no other network is set all containers will be able to communicate with each other throught this network.

Nonetheless, there are scenarios in which you may want to create a separate network for a set of containers. For example, if you have a three tier application with a frontend, backend and database. In this scenario you want the frontend to communicate with the backend but not with your database container directly as it will be using the backend for that.

To achieve that you can create a "front" and a "back" network, attach the frontend and the backend container to the "front" network and attach the backend and the database containers to the "back" network. With this setup the backend container will be able to communicate with the frontend and the database as it's attached to both networks, but the fronend and database containers wont see each other.

This is represented in the following picture:

![Multiplenetworks](Pictures/multiple-networks.png)

Lets see Podman networks in practice. First log into the server:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman201'] }}

Create the network

In [ ]:
podman network create example-student{{ STDID }}

With the previous command we have created a Podman network called "example-1", the command has options to configure it (gateway address, subnet mask, IPv4 or IPv6, etc). We can now use the following command to list all the networks we have:

In [ ]:
podman network ls

As you can see we have our newly created "example-1" network and the default "podman" network available. We can remove our recently created network by running:

In [ ]:
podman network rm example-student{{ STDID }}

Check if it was deleted:

In [ ]:
podman network ls

You can also run the following command to remove all the unused networks at once:

In [ ]:
podman network prune -f

Remember that Podman will attach a container to the default network when the network is not specified in the podman run command. This is important because the default network uses slirp4netns network mode while the ones you create manually will use the bridge network mode. If certain container is using slirp4netns network mode Podman wont be able to connect it to a bridged network and the command will fail.

Moreover, the default Podman network has the domain name system (DNS) resolution for other containers disabled. If you want to use DNS resolution between your containers create a dedicated network. When DNS is enabled the hostname for a container is the same as the container name.

> **Note**: There are other components, like firewall rules, that could be preventing the connection between containers in the same network.

Let's see a simple example of Podman networking in action. We are going to create two containers that will represent the frontend and backend containers of a simple application. First we will create both of them in separate networks and we will see how they're not able to connect.

Create the two podman networks:

In [ ]:
podman network create frontend-network
podman network create backend-network

Check all of them exist:

In [ ]:
podman network ls

Next create the frontend and backend container. We will be using nginx containers because it makes it easier to test connectivity by running a single curl command, in a real world scenario the containers would be different.

In [ ]:
podman run -d --rm \
  --network frontend-network \
  --name frontend-container \
  docker.io/library/nginx:latest
podman run -d --rm \
  --network backend-network \
  --name backend-container \
  docker.io/library/nginx:latest

Check the connection between containers by executing curl and trying to reach the other containers using their DNS names which is the same as their container name.
Try to reach the backend container from the frontend container:

In [ ]:
podman ps

In [ ]:
podman exec frontend-container curl -s backend-container

No output, just as expected because they are sitting in different networks. We should get the same output if we try to reach the frontend from the backend container.

In [ ]:
podman exec backend-container curl -s frontend-container

Again, we got nothing as expected. If we want them to reach each other they need to be in the same network. The same container can sit in several networks at the same time so we will use the following command to connect the backend container to the frontend network:

In [ ]:
podman network connect frontend-network backend-container

Now it's connected to the frontend network, so the frontend should be able to reach the backend.

In [ ]:
podman exec frontend-container curl -s backend-container

It was able to communicate as expected. And the backend should be able to reach the frontend as well:

In [ ]:
podman exec backend-container curl -s frontend-container

With this simple exercise we have demonstrated how simple it is to manage multiple networks and DNS resolution in a Podman environment.

# Get to know our Patient Portal application

During this workshop we will work with the Patient Portal application. This application is used as an example application for different open source projects focused on microservices such as [Skupper](skupper.io).

The application is an online portal for patients to book time with doctors. It has three micro services, that will be three containers in our Podman environment. It looks as shown in the following image:

![PatientPortalApplication](Pictures/patient-portal-application.png)

As you can see, users access the frontend microservice. The frontend needs a database for storing the appointments data and, as users can pay through the portal, the payment processing is separated in a third microservice. It's important to note that the frontend needs to reach the payment processor and the database microservices but the database and the payment processor do not need to reach each other.

# Use Podman networks with Patient Portal application

We are going to guarantee that the database and the payment processor microservices cannot reach each other. This way we guarantee there is not unwanted traffic and decrease the attack vector for malware. For that we will create two networks as represented in the following picture:

![PatientPortalApplication](Pictures/patient-portal-application-network.png)

The frontend container will be sitting in both networks, payments and database, while the other two microservics will be limited to a single network. Let's create the networks first:

In [ ]:
podman network create database
podman network create payment
podman network ls

Our networks have been created and both of them have DNS enabled by default. Just to be sure, we can check if it's enabled by running the following command:

In [ ]:
podman network inspect payment --format "Network: {{ '{{' }} .Name {{ '}}' }}, DNS Enabled: {{ '{{' }} .DNSEnabled {{ '}}' }}"
podman network inspect database --format "Network: {{ '{{' }} .Name {{ '}}' }}, DNS Enabled: {{ '{{' }} .DNSEnabled {{ '}}' }}"

Now we know for sure that DNS is enabled in both networks. Then lets deploy our containers starting with the database in the database network.

In [ ]:
podman run -d --rm --name database --network database quay.io/skupper/patient-portal-database

> **Important note**: the database inside the container needs to be initialized before deploying the frontend container. Be sure you wait a few seconds after deploying the database so it has enough time to initialize.

The payment-processor container in the payment network:

In [ ]:
podman run -d --rm --name payment-processor --network payment quay.io/skupper/patient-portal-payment-processor

And, last, our frontend in the payment network:

In [ ]:
podman run -d --rm --name frontend --network payment -p 8080:8080 \
-e DATABASE_SERVICE_HOST="database" \
-e DATABASE_SERVICE_PORT="5432" \
-e PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor" \
-e PAYMENT_PROCESSOR_SERVICE_PORT="8080" \
quay.io/skupper/patient-portal-frontend

Note that we pass to the container a few environment variables so it knows the data to be used when connecting to the other two containers. We gave the "database" and "payment-processor" as hostnames because Podman DNS resolution uses the container names as DNS names.

If you've paid attention you may have realized that our container is only connected to one network, it should not be able to reach the database. Lets check the logs:

In [ ]:
podman logs frontend

Just as expected it cannot connect to the database. It has to connect to the database at boot so we need to delete it:

In [ ]:
podman stop frontend

Simply recreate it adding the database network and it should work:

In [ ]:
podman run -d --rm --name frontend --network payment,database -p 8080:8080 \
-e DATABASE_SERVICE_HOST="database" \
-e DATABASE_SERVICE_PORT="5432" \
-e PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor" \
-e PAYMENT_PROCESSOR_SERVICE_PORT="8080" \
quay.io/skupper/patient-portal-frontend

Review logs:

In [ ]:
podman logs frontend

It doesn't complain about connection anymore, it's working. Check you can access the frontend microservice via http:

In [ ]:
curl -s localhost:8080

Everything works as expected!

With this we've seen a real example of how to use Podman networks to isolate communication.

# Cleanup

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f

In [ ]:
%logout 16.31.86.106

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Lab 2 : Podman Storage

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="2-WKSHP-Podman-storage.ipynb" target="New" title="Next LAB: Podman storage"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="0-ReadMeFirst.ipynb" target="New" title="Back: Introduction to Podman"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="2-WKSHP-Podman-storage.ipynb" target="New" title="Next:Podman Storage"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
